In [ ]:
import zipfile
with zipfile.ZipFile("ChoquetClass.zip", "r") as zip_ref:
    zip_ref.extractall("ChoquetClass")


In [ ]:
import ChoquetClass as cc
import pandas as pd
from ChoquetClass import classifier


#cargamos nuestros datos y reescribimos los nombres de los atributos
df=pd.read_csv("iris.dat")
df.columns = ["atributo1", "atributo2", "atributo3", "atributo4", "Class"]

# divide el dataset en training(70%) y test (30%),
# Nos devuelve los valores mínimos y máximos de cada atributo
# Pide al usuario especificar la columna de la clase
train_Data, test_Data, class_column, Values  = cc.analyse.preparacion(df)

#crea automáticamente 3 conjuntos difusos para cada atributo con funciones triangulares
#crea las variables linguisticas para cada conjunto difuso
variables=cc.CrearVariablesLinguisticas.crear(Values)

#generación de reglas fuzzy automáticas en base al algoritmo genético
P_best, q_best, historial = cc.rules_generator.algoritmo_genetico(
    train_data=train_Data,
    test_data=test_Data,
    col_clase="Class",
    variables=variables,
    num_rules=20,
    num_iter=5,
    TQ=15)



Columnas disponibles en el DataFrame:
['atributo1', 'atributo2', 'atributo3', 'atributo4', 'Class']

📊 DATA DE ENTRENAMIENTO (primeras filas):
     atributo1  atributo2  atributo3  atributo4            Class
88         5.5        2.5        4.0        1.3  Iris-versicolor
25         5.0        3.4        1.6        0.4      Iris-setosa
138        6.9        3.1        5.4        2.1   Iris-virginica
29         4.8        3.1        1.6        0.2      Iris-setosa
19         5.4        3.4        1.7        0.2      Iris-setosa

✅ Valores Mínimos y Máximos por atributo (sin incluir la clase):
atributo1: min=4.3, max=7.9
atributo2: min=2.0, max=4.2
atributo3: min=1.1, max=6.9
atributo4: min=0.1, max=2.5
Iteración inicial: CR = 0.8222, q_used = [1.0, 1.0, 1.0]
TQ 1: Nuevo mejor CR = 0.8667, q_used = [1.3372320308851275, 0.0656444378654451, 12.129802239790191]

--- Iteración 1 ---
CR con q_best_used: 0.8889
Iter 1: ¡Nueva mejor Población! CR = 0.8889

--- Iteración 2 ---
CR con q_best_used

In [ ]:
clf = cc.classifier.Classifier(variables, P_best, train_Data, class_column, q_vector=q_best)

# Un patrón de prueba (fila del dataset o diccionario)
patron = test_Data.iloc[3]

clase_predicha, resultado = clf.classify_Choquet(patron)
print("Clase predicha:", clase_predicha)

Clase predicha: Iris-versicolor


In [ ]:
print(patron)

atributo1                6.0
atributo2                3.4
atributo3                4.5
atributo4                1.6
Class        Iris-versicolor
Name: 84, dtype: object


In [ ]:
#funciones de pre-agregación


#1 cópula tipo Choquet

# Crear instancia (usa las variables lingüísticas que ya tengas definidas)
asociacion_fuzzy = cc.rules_generator.Asociacion_fuzzy(variables)

# Llamar correctamente al método 'calcular'
asociation = asociacion_fuzzy.calcular(
    reglas_f=P_best,
    ejemplo=patron,
    data=df,
    col_clase='Class'
)

print(asociation)



[{'Iris-virginica': np.float64(0.11956032702798094)}, {'Iris-versicolor': np.float64(-0.19704847870703304)}, {'Iris-setosa': 0.0}, {'Iris-virginica': np.float64(-0.28033245986078453)}, {'Iris-setosa': np.float64(-0.17241379310344826)}, {'Iris-virginica': 0.0}, {'Iris-virginica': np.float64(0.11956032702798094)}, {'Iris-versicolor': 0.0}, {'Iris-setosa': 0.0}, {'Iris-virginica': 0.0}, {'Iris-setosa': 0.0}, {'Iris-virginica': np.float64(0.11956032702798094)}, {'Iris-virginica': 0.0}, {'Iris-virginica': np.float64(0.06430120747127208)}, {'Iris-virginica': 0.0}, {'Iris-setosa': 0.0}, {'Iris-setosa': 0.0}, {'Iris-setosa': 0.0}, {'Iris-versicolor': np.float64(0.1606036865944474)}, {'Iris-setosa': 0.0}]


In [ ]:
from importlib import reload
reload(classifier)
reload(cc.aggregation_functions)
reload(cc.algorithm_genetic)


<module 'ChoquetClass.algorithm_genetic' from '/content/ChoquetClass/algorithm_genetic.py'>

In [ ]:
#A partir de aquí probaremos el optimizador genético

taza_clasificacion = cc.algorithm_genetic.CR_CC_choquet(
    test_Data,
    train_Data,
    P_best,
    variables,
    col_clase=class_column,
    q_vector=q_best
)
print(taza_clasificacion)




0.8666666666666667


In [ ]:
print(class_column)

Class


In [ ]:
cc.algorithm_genetic.optimizar_q_CC_choquet(
    test_data=test_Data,
    train_data=train_Data,
    reglas=P_best,
    variables=variables,
    col_clase=class_column,
    q_inicial=[1,1,1],
    num_iter=10
)


Iteración 0: CR = 0.7556, q = [1, 1, 1]
Iteración 10: Nuevo mejor CR = 0.9333, q = [0.8314712782269702, 0.3330644527551036, 0.38685119686569636]


([0.8314712782269702, 0.3330644527551036, 0.38685119686569636],
 0.9333333333333333)

In [ ]:
cc.rules_generator._terms_for_attr(variables, 'atributo1')

['BAJO', 'MEDIO', 'ALTO']

In [ ]:
cc.rules_generator.SolidezChoquet(variables)

In [ ]:
best_pob, q_best, historial=cc.algorithm_genetic.algoritmo_genetico_comb_aditive(
    train_Data,
    test_Data,
    class_column,
    variables,
    num_rules=20,
    num_iter=3,
    TQ=3,
    cross_prob=0.9
)

Iteración inicial: CR = 0.7778, q_used = [1.0, 1.0, 1.0]

--- Iteración 1 ---
CR con q_best_used: 0.8667
Iter 1: ¡Nueva mejor población! CR = 0.8667

--- Iteración 2 ---
CR con q_best_used: 0.8000

--- Iteración 3 ---
CR con q_best_used: 0.8667

Mejor CR final: 0.8667
Mejor q_used encontrado: [1.0, 1.0, 1.0]


In [ ]:
print(historial)

[(0, 0.7777777777777778, [1.0, 1.0, 1.0]), ('TQ', 0.7777777777777778, [1.0, 1.0, 1.0]), (1, 0.8666666666666667, [1.0, 1.0, 1.0]), (2, 0.8666666666666667, [1.0, 1.0, 1.0]), (3, 0.8666666666666667, [1.0, 1.0, 1.0])]


In [28]:
best_pob, q_best, historial=cc.algorithm_genetic.algoritmo_genetico_CC_Choquet(
    train_Data,
    test_Data,
    class_column,
    variables,
    num_rules=20,
    num_iter=3,
    TQ=3,
    cross_prob=0.9
)

Iteración 0: CR = 0.6222, q = [1.0, 1.0, 1.0]
Iteración 2: Nuevo mejor CR = 0.6444, q = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]
Optimización inicial completada: CR = 0.6444, q_used = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]

--- Iteración 1 ---
Iteración 0: CR = 0.8889, q = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]
Iter 1: CR optimizado = 0.8889, q_used = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]
✅ Iter 1: Nueva mejor población, CR = 0.8889

--- Iteración 2 ---
Iteración 0: CR = 0.8667, q = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]
Iter 2: CR optimizado = 0.8667, q_used = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]

--- Iteración 3 ---
Iteración 0: CR = 0.8889, q = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]
Iter 3: CR optimizado = 0.8889, q_used = [0.5712712702345955, 0.8017269947539812, 0.43320120293012526]

🏁 Mejor CR final: 0.8889
Mejor q_used e

In [29]:
clf = cc.classifier.Classifier(variables, best_pob, train_Data, class_column, q_vector=q_best)

# Un patrón de prueba (fila del dataset o diccionario)
patron = test_Data.iloc[3]

clase_predicha, resultado = clf.classify_CC_Choquet(patron)
print("Clase predicha:", clase_predicha)

Clase predicha: Iris-versicolor
